## V1

In [142]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import requests
from urllib.parse import parse_qs, urlparse
from serpapi import GoogleSearch as search
from tqdm import tqdm
from timeout_decorator import timeout
import json
import shutil
import os 
import pandas as pd
from goose3 import Goose
import ssl
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse, parse_qs
from serpapi import GoogleSearch
import undetected_chromedriver.v2 as uc
from selenium.webdriver.common.by import By
import sys



In [12]:
data_lacunaire = pd.read_excel('/Users/hugo.cadet/PycharmProjects/Meltwater_tag.xlsx')

In [13]:
data = pd.read_excel('/Users/hugo.cadet/Downloads/Copie de 20241202_Mohammed_bin_Salman_Meltwater data excluding Twitter.xlsx')

In [14]:
data = data.merge(data_lacunaire[['Document ID','Scrapped']], on='Document ID', how='left')

In [15]:
data_ini = data.copy()
data = data.dropna(subset=['URL'])

In [16]:
data_false = data[data['Scrapped'] == False]
reach_by_domain = data_false.groupby('Source Domain')['Reach'].mean().sort_values(ascending=False)

In [17]:
data_true = data[data['Scrapped'] == True]

In [18]:
p1 = (data_false.groupby('Source Domain').size() / len(data_false)).reset_index()
p1.columns = ['Source Domain', 'Share of Unscraped Articles per Source Among All Unscraped Articles']

p1['Number of Unscraped Articles'] = data_false['Source Domain'].value_counts().reindex(p1['Source Domain']).values

reach_per_source = (
    data_false.groupby('Source Domain')['Reach']
    .sum()
    .reset_index()
    .rename(columns={'Reach': 'reach'})
)
p1 = p1.merge(reach_per_source, on='Source Domain', how='left')

false_counts = data_false['Source Domain'].value_counts()
total_counts = data['Source Domain'].value_counts()
proportion_total = (false_counts / total_counts).reset_index()
proportion_total.columns = ['Source Domain', 'Unscraped Articles Share per Source']

excel_final = p1.merge(proportion_total, on='Source Domain', how='left')

excel_final = excel_final[
    [
        'Source Domain',
        'reach',
        'Number of Unscraped Articles',
        'Share of Unscraped Articles per Source Among All Unscraped Articles',
        'Unscraped Articles Share per Source'
    ]
]

### msn.com

In [19]:
msn = data_false[data_false['Source Domain'] == 'msn.com']

In [86]:
#### Final Version ####
## goose non possible
# Obligé de récup l'id dans l'url puis de l'utiliser dans l'API

def get_ids(msn_urls): 
    ids = []
    for url in msn_urls:
        url_parts = str(url).split("-")  
        last_part = url_parts[-1]
        if len(last_part) == 8:
            ids.append(last_part)
        elif len(last_part) == 17:
            ids.append(last_part[:-9])  # Enlève le suffixe "#comment"
        elif len(last_part) == 5 and len(url_parts) >= 3:
            ids.append(url_parts[-3][:8])
        else:
            ids.append(None)
    return ids


def msn_scraper(msn, is_url=False, urls=None):
    contents = []
    raw_content = []

    if is_url:  
        urls = urls
    else: 
        urls = msn['URL'].tolist()

    ids = get_ids(urls)

    for i, id_ in tqdm(enumerate(ids), desc='Processing msn URLs'):
        if id_ is None:
            print(f"ID manquant pour URL : {urls[i]}")
            row_id = msn['Document ID'].iloc[i]
            article_data = {
                "row_id": row_id,      
                'data': {
                    'meta': {'canonical': urls[i]},
                    'image': None,
                    'domain': None,
                    'title': None,
                    'cleaned_text': None,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None
                }
            }
            contents.append(article_data)
            raw_content.append(None)
            continue
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
            'Referer': 'https://www.msn.com/',
        }

        try:
            # Faire une requête pour récupérer les données de l'API
            response = requests.get(
                f'https://assets.msn.com/content/view/v2/Detail/en-us/{id_}', 
                headers=headers, timeout=10
            )
            raw_content.append(response.text)
            response.raise_for_status()  # Vérifie si la requête a réussi (status code 200)

            # Extraire les données JSON et le contenu HTML
            data = response.json()
            title = data.get('title', '')
            body_html = data.get('body', '')
            
            # Utilisation de BeautifulSoup pour extraire le texte
            soup = BeautifulSoup(body_html, "html.parser")
            content_text = soup.get_text(separator="\n")  # Récupérer le texte

            # Ajouter les données extraites dans le résultat
            row_id = msn['Document ID'].iloc[i]
            article_data = {
                "row_id": row_id,      
                'data': {
                    'meta': {'canonical': urls[i]},
                    'image': None,  # Vous pouvez ajouter une logique pour récupérer une image si nécessaire
                    'domain': None,  # Vous pouvez extraire le domaine si nécessaire
                    'title': title,
                    'cleaned_text': content_text,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None  # Vous pouvez extraire la date de publication si nécessaire
                }
            }

            contents.append(article_data)

        except requests.exceptions.RequestException as e:
            print(f"Erreur pour l'ID {id_} : {e}")
            # En cas d'erreur, ajouter un article avec des valeurs par défaut
            row_id = msn['Document ID'].iloc[i]
            article_data = {
                "row_id": row_id,
                'data': {
                    'meta': {'canonical': urls[i]},
                    'image': None,
                    'domain': None,
                    'title': None,
                    'cleaned_text': None,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None
                }
            }
            contents.append(article_data)

        # Ajout d'un délai pour éviter de surcharger le serveur
        time.sleep(0.5)  

    return contents, raw_content

### Sauress.com 

In [21]:
sauress = data_false[data_false['Source Domain'] == 'sauress.com']

In [83]:
def sauress_scraper(sauress):
    contents_sauress, raw_content = [], []
    
    for i, url in tqdm(enumerate(sauress['URL']), total=len(sauress['URL']), desc="Processing sauress URLs"):
        time.sleep(0.5)
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
                'Referer': 'https://www.msn.com/',
            }
            response = requests.get(url, timeout=10, headers=headers)
            raw_content.append(response.text)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                
                # Vérification du titre de la page
                title = soup.title.string.strip() if soup.title and soup.title.string else ""
                if not title:
                    og_title = soup.find("meta", property="og:title")
                    if og_title and og_title.get("content"):
                        title = og_title["content"].strip()  
            else:
                print("Page non disponible ou code HTTP différent de 200")
                title = "Titre non disponible"
            
            # Extraction du texte principal
            spans = soup.find_all("span", class_="articlecontent")
            row_id = sauress['Document ID'].iloc[i]
            
            # Construction des données de l'article
            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': url},
                    'image': None,  # Ajouter la logique pour récupérer l'image si nécessaire
                    'domain': None,  # Ajouter la logique pour récupérer le domaine si nécessaire
                    'title': title,
                    'cleaned_text': ', '.join([span.text.strip() for span in spans]),
                    'opengraph': {},  # Ajouter les données OpenGraph si nécessaire
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None  # Ajouter la logique pour récupérer la date de publication si nécessaire
                }
            }

            contents_sauress.append(article_data)

        except requests.exceptions.RequestException as e:
            print(f"\n🔗 {url}")
            print(f"Erreur de requête : {e}")
            # Ajouter des données par défaut en cas d'erreur
            row_id = sauress['Document ID'].iloc[i]
            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': url},
                    'image': None,
                    'domain': None,
                    'title': None,
                    'cleaned_text': None,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None
                }
            }
            contents_sauress.append(article_data)

    return contents_sauress, raw_content


### shafaqna.com	

In [23]:
#Ok, scraper l'article avant la div latest news, si il n'y a pas d'article avant cela veut dire qu'il n'est plus dispo

In [24]:
shafaqna = data_false[data_false['Source Domain'] == 'shafaqna.com']
shafaqna['URL'].apply(lambda x : x[:-7]).unique()
#shafaqna['URL'].apply(lambda x : x[-7:]).tolist()
# Meme format : 'https://egypt.shafaqna.com/../ + int


array(['https://egypt.shafaqna.com/AR/AL/',
       'https://qatar.shafaqna.com/AR/AL/',
       'http://malaysia.shafaqna.com/MY/AL/',
       'https://algeria.shafaqna.com/', 'https://uae.shafaqna.com/AR/AL/',
       'https://indonesia.shafaqna.com/ID/AL/',
       'http://kenya.shafaqna.com/EN/A',
       'https://thailand.shafaqna.com/TH/AL/',
       'http://zimbabwe.shafaqna.com/EN/AL/',
       'https://venezuela.shafaqna.com/ES/AL/',
       'http://bahrain.shafaqna.com/AR/AL',
       'http://poland.shafaqna.com/PL/AL/',
       'http://taiwan.shafaqna.com/CN/AL/',
       'http://chile.shafaqna.com/ES/AL/',
       'https://www.pk.shafaqna.com/EN/AL/',
       'https://singapore.shafaqna.com/EN/AL/',
       'https://fa.shafaqna.com/news/1731394/%d8%b9%d8%b1%d8%a8%d8%b3%d8%aa%d8%a7%d9%86-%d8%a8%db%8c%d8%b4-%d8%a7%d8%b2-%db%b8%db%b5-%d8%af%d8%b1%d8%b5%d8%af-%d8%a7%d8%b2-%d8%aa%d9%88%d8%b5%db%8c%d9%87-%d9%87%d8%a7%db%8c-%d8%b3',
       'https://algeria.shafaqna.com/AR/AL/',
       'http://ma

In [73]:
def shafaqna_scraper(shafaqna):
    content, raw_content = [], []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
        'Referer': 'https://www.msn.com/',
    }

    for i, url in tqdm(enumerate(shafaqna['URL']), desc='Processing shafaqna URLs'):
        row_id = shafaqna['Document ID'].iloc[i]
        article_none = {
            'row_id': row_id,
            'data': {
                'meta': {'canonical': url},
                'image': None,
                'domain': None,
                'title': None,
                'cleaned_text': None,
                'opengraph': {},
                'tags': [],
                'tweets': [],
                'movies': [],
                'links': [],
                'authors': [],
                'publish_date': None
            }
        }
        try:
            response = requests.get(url, timeout=10, verify=False, headers=headers)
            raw_content.append(response.text)

            # Vérification du code de statut
            if response.status_code != 200:
                print(f"Page non disponible ou code HTTP différent de 200 : {url}")
                
                content.append(article_none)
                continue

            # Extraire le contenu principal de la page
            soup = BeautifulSoup(response.text, "html.parser")
            news = soup.find("div", id="right-column")
            news = news.find("div", id="news-text") if news else None

            if not news:
                print(f"No news found in the url number {i} : {url}")
                
                content.append(article_none)
                continue

            first_non_newsbox_div = news.find(
                lambda tag: (
                    tag.name == "div"
                    and tag.get("class") != ["news", "box"]
                    and tag.get("id") != "titlebar"
                )
            )

            if first_non_newsbox_div:
                href = first_non_newsbox_div.find('a')['href']

                # Faire une requête pour l'article lié
                article_html = requests.get(href, timeout=10).text

                # Extraire les données de l'article avec Goose
                g = Goose()
                article = g.extract(raw_html=article_html)

                row_id = shafaqna['Document ID'].iloc[i]
                article_data = {
                    'row_id': row_id,
                    'data': {
                        'meta': {'meta_description': article.meta_description, 'url': url},
                        'image': article.top_image.src if article.top_image else None,
                        'domain': article.domain,
                        'title': article.title,
                        'cleaned_text': article.cleaned_text,
                        'opengraph': article.opengraph,
                        'tags': article.tags,
                        'tweets': article.tweets,
                        'movies': article.movies,
                        'links': article.links,
                        'authors': article.authors,
                        'publish_date': str(article.publish_date) if article.publish_date else None
                    }
                }
                content.append(article_data)
            else:
                print(f"No suitable content found in the url number {i} : {url}")
                
                content.append(article_none)

        except requests.exceptions.RequestException as e:
            print(f"Erreur pour {url}: {e}")
            # En cas d'erreur de requête, ajouter un article vide
            
            content.append(article_none)

        # Délai pour éviter de surcharger le serveur
        time.sleep(0.5)

    return content, raw_content


### Alriyadh

In [85]:
def open_url_no_verify(url: str):
    """Ouvre une URL sans vérifier le certificat SSL (à éviter en production)."""
    context = ssl._create_unverified_context()
    return urlopen(url, context=context)

def alriyadh_scraper(alri):
    contents_alri, raw_content = [], []
    
    for i, url in tqdm(enumerate(alri['URL']), desc="Processing Alriyadh URLs"):
        try:
            response = open_url_no_verify(url)
            html = response.read().decode('utf-8', errors='ignore')
            raw_content.append(html)
            
            soup = BeautifulSoup(html, 'html.parser')
            title = soup.find('div', class_='article-title').text if soup.find('div', class_='article-title') else 'Titre non trouvé'
            text = soup.find('div', class_='article-text').text if soup.find('div', class_='article-text') else 'Texte non trouvé'
            
            row_id = alri['Document ID'].iloc[i]

            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': url},
                    'image': None,  # À ajuster si une image est présente sur la page
                    'domain': None,  # Vous pouvez extraire le domaine si nécessaire
                    'title': title,
                    'cleaned_text': text,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None  # Vous pouvez extraire la date de publication si nécessaire
                }
            }
            contents_alri.append(article_data)  

        except (HTTPError, URLError, ssl.SSLError, ValueError) as e:
            print(f"Erreur pour {url} : {e}")
            # Pour ne pas décaler les id, on rajoute quand même des valeurs par défaut (None)
            row_id = alri['Document ID'].iloc[i]
            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': url},
                    'image': None,
                    'domain': None,
                    'title': None,
                    'cleaned_text': None,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None
                }
            }
            contents_alri.append(article_data)
        time.sleep(0.5)
    
    return contents_alri, raw_content


In [27]:
"""alri = data_false[data_false['Source Domain'] == 'alriyadh.com']
contents_alri = alriyadh_scraper(alri[:5])"""

"alri = data_false[data_false['Source Domain'] == 'alriyadh.com']\ncontents_alri = alriyadh_scraper(alri[:5])"

### sahafaa

In [28]:
#Pas possible en requests, pas d'appel api  ==> selenium
# Ne marche pas avec goose ..

In [69]:
def extract_article_text(html, url):
    soup = BeautifulSoup(html, "html.parser")

    # Liste des classes possibles à tester dans des <div> -> agregateur donc pas forcement tout le tps la meme structure
    class_candidates = [
        'text-body',
        'story_text',
        'post--content',
        'post_details_inner',
        "news-content",
        "news_text",
        "article",
        "description-text",
        'col-md-12',
    ]

    content_parts = []

    for class_name in class_candidates:
        if class_name == 'article':
            found = soup.find_all('div', id=lambda c: c and 'article' in c)  # seul cas ou le texte est dans un div avec id (sinon class)
        else:
            found = soup.find_all('div', class_=lambda c: c and class_name in c)

        if found and str(404) not in str(found):
            for f in found:
                text = f.get_text(strip=True)
                if len(text) > 100:  # Ajout d'un contrôle sur la taille du texte
                    content_parts.append(text)

            if content_parts:
                return {
                    'content': "##".join(content_parts),
                    'Title': soup.find('meta', property='og:title')['content'] if soup.find('meta', property='og:title') else "Titre non trouvé",
                    'URL': url
                }

    print(f"Aucun contenu trouvé dans les classes spécifiées pour l'url {url} ... \n On cherche dans le og:description")
    meta = soup.find('meta', property='og:description')
    if meta and meta.get('content') and '[…]' not in meta['content']:
        return {
            'content': meta['content'],
            'title': soup.find('meta', property='og:title')['content'] if soup.find('meta', property='og:title') else "Titre non trouvé",
            'url': url
        }

    print(f"##### Aucun contenu trouvé pour l'url {url} ###### \n")
    return ""


def sahaafa_scraper(sahafaa):
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--ignore-ssl-errors=yes')
    options.add_argument('--ignore-certificate-errors')
    # options.add_argument('--headless')  

    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(30)

    contents, raw_content = [], []
    
    for i, url in tqdm(enumerate(sahafaa['URL'].tolist()), desc="Processing sahaafa.net URLs"):
        row_id = sahafaa['Document ID'].iloc[i]
        article_none = {
            'row_id': row_id,
            'data': {
                'meta': {'canonical': url},
                'image': None,
                'domain': None,
                'title': None,
                'cleaned_text': None,
                'opengraph': {},
                'tags': [],
                'tweets': [],
                'movies': [],
                'links': [],
                'authors': [],
                'publish_date': None
            }
        }
        try:
            driver.get(url)
            
            # Essayer de passer dans un iframe si présent
            try:
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "iframe"))
                )
                driver.switch_to.frame(iframe)
            except Exception as e:
                contents.append(article_none)
                print(f"[{url}] Pas d'iframe trouvé ou erreur :", e)

            html = driver.page_source
            raw_content.append(html)
            soup = BeautifulSoup(html, "html.parser")
            elements = extract_article_text(html, url)

            # Récupérer l'ID du document
            row_id = sahafaa['Document ID'].iloc[i]

            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': elements['URL']},
                    'image': None,  # Vous pouvez extraire une image si nécessaire
                    'domain': None,  # Vous pouvez extraire le domaine si nécessaire
                    'title': elements["Title"],
                    'cleaned_text': elements["content"],
                    'opengraph': {},  # Vous pouvez compléter selon votre besoin
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None  # Vous pouvez extraire la date de publication si nécessaire
                }
            }
            contents.append(article_data)
            driver.switch_to.default_content()  # Revenir au contenu principal

        except Exception as e:
            print(f"[{url}] Erreur générale :", e)
            # Pour ne pas décaler les ID, on rajoute quand même des valeurs par défaut (None)
            contents.append(article_none)

        time.sleep(0.5)

    driver.quit()
    
    return contents, raw_content


### klyoum.com	

In [ ]:
def klyoum_scraper(klyoum):
    g = Goose()
    contents, raw_content = [], []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
        'Referer': 'https://www.msn.com/',
    }

    for i, url in tqdm(enumerate(klyoum['URL'].tolist()),desc='processing klyoum URLs'):
        row_id = klyoum['Document ID'].iloc[i]
        
        # Article par défaut en cas d'erreur pour ne pas décaler les ids
        article_none = {
            'row_id': row_id,
            'data': {
                'meta': {'canonical': url},
                'image': None,
                'domain': None,
                'title': None,
                'cleaned_text': None,
                'opengraph': {},
                'tags': [],
                'tweets': [],
                'movies': [],
                'links': [],
                'authors': [],
                'publish_date': None
            }
        }
        
        try:
            response = requests.get(url, timeout=10, headers=headers)
            response.raise_for_status()
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")
            contents.append(article_none)
            raw_content.append(None)
            continue
        
        raw_content.append(response.text)
        soup = BeautifulSoup(response.text, 'html.parser')
        button = soup.find('a', attrs={
            'onclick': "gtag('event', 'click', {'event_category': 'link to original website','event_label': 'button_link to original website'});"
        })

        if button and button.get('href'):
            href = button.get('href')
            params = parse_qs(urlparse(href).query)
            domain = params.get('cc', [''])[0]
            title = params.get('t', [''])[0]
            
            if domain and title:
                params = {
                    "api_key": "77c5ebb6d414ddeb58b73c1eccddfcddedb5ce0996bb1a68f5bbd419513d8088", #laura's one
                    "engine": "google",
                    "hl": "fr",
                    "q": f'{str(domain)} {str(title)}'  # La vraie requête ici
                }
                search = GoogleSearch(params)
                data = search.get_dict()

                try:
                    links = data['organic_results']
                except KeyError as e:
                    contents.append(article_none)
                    print(f"No organic value in the serpapi result for {url}")
                    continue
                
                try:
                    
                    if links:
                        first_link = links[0]["link"]
                        article = g.extract(first_link)
                        print("daaate", article.publish_date)
                        article_data = {
                            'row_id': row_id,
                            'data': {
                                'meta': {'meta_description': article.meta_description, 'url': first_link},
                                'image': article.top_image.src if article.top_image else None,
                                'domain': article.domain,
                                'title': article.title,
                                'cleaned_text': article.cleaned_text,
                                'opengraph': article.opengraph,
                                'tags': article.tags,
                                'tweets': article.tweets,
                                'movies': article.movies,
                                'links': article.links,
                                'authors': article.authors,
                                'publish_date': str(article.publish_date) if article.publish_date else None
                            }
                        }
                        contents.append(article_data)
                    else:
                        print("No Google result found for the URL.")
                        contents.append(article_none)

                except Exception as e:
                    print(f"Google Search failed for {url}: {e}")
                    contents.append(article_none)
            else:
                print(f"Missing parameters in href for {url}.")
                contents.append(article_none)
        else:
            print(f"No button found for {url}.")
            contents.append(article_none)

        time.sleep(0.3)

    return contents, raw_content


### SPA

In [31]:
spa = data_false[data_false['Source Domain'] == 'spa.gov.sa']

In [66]:
def spa_scraper(spa, timeout=150):
    contents, raw_content = [], []
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
        'Referer': 'https://www.msn.com/',
    }

    for i, url in tqdm(enumerate(spa['URL'].tolist()), desc="Processing spa URLs"):
        row_id = spa['Document ID'].iloc[i]
        article_none = {
            'row_id': row_id,
            'data': {
                'meta': {'canonical': url},
                'image': None,
                'domain': None,
                'title': None,
                'cleaned_text': None,
                'opengraph': {},
                'tags': [],
                'tweets': [],
                'movies': [],
                'links': [],
                'authors': [],
                'publish_date': None
            }
        }
        try:
            # Effectuer la requête HTTP
            response = requests.get(url, timeout=timeout, headers=headers)
            raw_content.append(response.text)
            
            # Analyser la page avec BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Extraction du contenu de l'article avec Goose
            g = Goose()
            article = g.extract(raw_html=response.text)
            
            title = article.title
            meta = article.meta_description
            
            # Récupérer le contenu à partir de la balise og:description
            content = soup.find("meta", property="og:description")['content'] if soup.find("meta", property="og:description") else ""
            if content == "":
                print(f"Aucun contenu trouvé dans la balise og:description pour l'URL : {url}")
                continue  # Passer à l'URL suivante si aucun contenu n'est trouvé
            
            # Créer un dictionnaire de données à ajouter à `contents`
            fields = [
                'meta', 'image', 'domain', 'title', 'cleaned_text', 'opengraph',
                'tags', 'tweets', 'movies', 'links', 'authors', 'publish_date'
            ]
            d = {field: "" for field in fields}
            d['title'] = title
            d['cleaned_text'] = content
            d['meta'] = meta
            
            # Récupérer l'ID du document
            row_id = spa['Document ID'].iloc[i]
            
            # Ajouter les données de l'article à la liste `contents`
            article_data = {
                'row_id': row_id,
                'data': d
            }
            contents.append(article_data)
            print(f"Contenu extrait pour {url}")

        except requests.exceptions.RequestException as e:
            print(f"\nErreur de requête pour {url}: {e}")
            # Pour ne pas décaler les ID, on rajoute quand même des valeurs par défaut (None)
            
            contents.append(article_data)

        except Exception as e:
            # Gestion d'autres erreurs
            print(f"Erreur d'extraction pour {url}: {e}")
            
            contents.append(article_none)

        # Ajouter un délai pour éviter de surcharger le serveur
        time.sleep(0.5)

    return contents, raw_content


### aawsat.com



In [64]:
def aawsat_scraper(aawsat):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/120.0.0.0 Safari/537.36"
        )
    }
    contents, raw_contents = [], []
    
    for i, url in tqdm(enumerate(aawsat['URL']), desc="Processing aawsat URLs"):
        row_id = aawsat['Document ID'].iloc[i]
        article_none = {
            'row_id': row_id,
            'data': {
                'meta': {'canonical': url},
                'image': None,
                'domain': None,
                'title': None,
                'cleaned_text': None,
                'opengraph': {},
                'tags': [],
                'tweets': [],
                'movies': [],
                'links': [],
                'authors': [],
                'publish_date': None
            }
        }
        try:
            response = requests.get(url, headers=headers, timeout=10)
            raw_contents.append(response.text)

            if response.status_code == 200:
                g = Goose()
                article = g.extract(raw_html=response.text)
                row_id = aawsat['Document ID'].iloc[i]
                
                article_data = {
                    'row_id': row_id,
                    'data': {
                        'meta': {'meta_description': article.meta_description, 'url': url},
                        'image': article.top_image.src if article.top_image else None,
                        'domain': article.domain,
                        'title': article.title,
                        'cleaned_text': article.cleaned_text,
                        'opengraph': article.opengraph,
                        'tags': article.tags,
                        'tweets': article.tweets,
                        'movies': article.movies,
                        'links': article.links,
                        'authors': article.authors,
                        'publish_date': str(article.publish_date) if article.publish_date else None
                    }
                }
                contents.append(article_data)

            else:
                print(f"Accès échoué à : {url} avec le code {response.status_code}")
                # Pour ne pas décaler les ID, on rajoute quand même None
                
                contents.append(article_none)

        except Exception as e:
            print(f"Erreur lors du traitement de {url}: {e}")
            
            contents.append(article_none)

        # Ajout d'un délai pour éviter de surcharger le serveur
        time.sleep(0.5)

    return contents, raw_contents

### RT.com & NYT & CNN & Yahoo

In [34]:
rt = data_false[data_false['Source Domain'] == 'rt.com']
nytimes = data_false[data_false['Source Domain'] == 'nytimes.com']
cnn = data_false[data_false['Source Domain'] == 'cnn.com']
yahoo = data_false[data_false['Source Domain'] == 'yahoo.com']

In [63]:
def rt_nyt_cnn_scraper(rt):
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    contents,raw_contents = [],[]
    for i,url in tqdm(enumerate(rt['URL'].tolist()), desc="Processing rt/nyt/cnn URLs"):
        row_id = rt['Document ID'].iloc[i]
        article_none = {
            'row_id' : row_id,
            'data' : {
            'meta': {'canonical': url},
            'image': None,
            'domain': None,
            'title': None,
            'cleaned_text': None,
            'opengraph': {},
            'tags': [],
            'tweets': [],
            'movies': [],
            'links': [],
            'authors': [],
            'publish_date': None
        }}

        try:
            response = requests.get(url, timeout=10, headers= headers)
            response.raise_for_status()  
            raw_contents.append(response.text)
            g = Goose()
            article = g.extract(raw_html=response.text)
            row_id = rt['Document ID'].iloc[i]
            article_data = {
                'row_id' : row_id,
                'data' : {
                    'meta': article.meta_description,
                    'image': article.top_image.src if article.top_image else None,
                    'domain': article.domain,
                    'title': article.title,
                    'cleaned_text': article.cleaned_text,
                    'opengraph': article.opengraph,
                    'tags': article.tags,
                    'tweets': article.tweets,
                    'movies': article.movies,
                    'links': article.links,
                    'authors': article.authors,
                    'publish_date': str(article.publish_date) if article.publish_date else None
                }
            }

            contents.append(article_data)

        except requests.exceptions.RequestException as e:
            print(f"\nErreur de requête pour {url} : {e}")
            #pour ne pas décaler les id on rajotue quand même None
            
            contents.append(article_none)
        except Exception as e:
            #pour ne pas décaler les id on rajotue quand même None
            contents.append(article_none)
            print(f"\nErreur d'extraction pour {url} : {e}")

        time.sleep(0.5) 

    return contents,raw_contents


#content = rt_nyt_cnn_scraper(cnn)

### Investing

In [36]:
investing = data_false[data_false['Source Domain'] == 'investing.com']

In [60]:
def investing_scraper(invest):
    contents, raw_content = [], []
    
    # Initialisation des options de Chrome
    options = webdriver.ChromeOptions()  # Pas de headless sinon première attente ne fonctionne pas
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--ignore-ssl-errors=yes')
    options.add_argument('--ignore-certificate-errors')

    for i,url in tqdm(enumerate(invest['URL'].tolist())):
        driver = webdriver.Chrome(options=options)
        try:
            driver.get(url)

            # Attendre que le titre de la page soit chargé, il y a un délai de 3s pour charger la page 
            WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.TAG_NAME, "title")))

            raw_content.append(driver.page_source)
            element = driver.find_element(By.TAG_NAME, 'h1')

            html = driver.page_source
            g = Goose()
            article = g.extract(raw_html=html)
            row_id = invest['Document ID'].iloc[i]
            article_data = {
                'row_id': row_id,
                'data':{
                'meta': article.meta_description,
                'image': article.top_image.src if article.top_image else None,
                'domain': article.domain,
                'title': article.title,
                'cleaned_text': article.cleaned_text,
                'opengraph': article.opengraph,
                'tags': article.tags,
                'tweets': article.tweets,
                'movies': article.movies,
                'links': article.links,
                'authors': article.authors,
                'publish_date': str(article.publish_date) if article.publish_date else None
            }}

            contents.append(article_data)

        except Exception as e:
            print(f"Une erreur est survenue pour l'URL {url}: {e}")
            #pour ne pas décaler les id on rajotue quand même None
            row_id = invest['Document ID'].iloc[i]
            article_data = {
                'row_id' : row_id,
                'data' : {
                'meta': {'canonical': url},
                'image': None,
                'domain': None,
                'title': None,
                'cleaned_text': None,
                'opengraph': {},
                'tags': [],
                'tweets': [],
                'movies': [],
                'links': [],
                'authors': [],
                'publish_date': None
            }}

            contents.append(article_data)

        finally:
            driver.quit() #obliger de le fermer à chaque it 

    return contents, raw_content

### Sohu.com

In [38]:
sohu = data_false[data_false['Source Domain'] == 'sohu.com']
#goose ne marche pas

In [84]:
def sohu_scraper(sohu):
    contents, raw_contents = [], []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    appended = False
    for i, url in tqdm(enumerate(sohu['URL'].tolist()), desc="Processing sohu URLs"):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            raw_contents.append(response.text)
            appended = True
            soup = BeautifulSoup(response.text, "html.parser")

            # Récupération du titre
            title = soup.title.string.strip() if soup.title and soup.title.string else ""

            # Extraction du contenu de l'article
            content_div = soup.find('div', attrs={'data-spm': 'content'})
            content = content_div.get_text(strip=True, separator="\n").strip() if content_div else ""

            row_id = sohu['Document ID'].iloc[i]
            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': url},
                    'image': None,
                    'domain': None,
                    'title': title,
                    'cleaned_text': content,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None
                }
            }

            contents.append(article_data)

        except requests.exceptions.RequestException as e:
            print(f"Erreur de requête pour {url} : {e}")
            # Pour ne pas décaler les ID, on ajoute quand même un article avec des valeurs par défaut (None)
            row_id = sohu['Document ID'].iloc[i]
            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': url},
                    'image': None,
                    'domain': None,
                    'title': None,
                    'cleaned_text': None,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None
                }
            }
            if not appended : raw_contents.append(None)
            contents.append(article_data)

        except Exception as e:
            print(f"Erreur d'extraction pour {url} : {e}")
            # Pour ne pas décaler les ID, on ajoute quand même un article avec des valeurs par défaut (None)
            row_id = sohu['Document ID'].iloc[i]
            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': url},
                    'image': None,
                    'domain': None,
                    'title': None,
                    'cleaned_text': None,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None
                }
            }
            if not appended : raw_contents.append(None)
            contents.append(article_data)

        # Ajout d'un délai pour éviter de surcharger le serveur
        time.sleep(0.5)

    return contents, raw_contents

### Goal 

In [57]:
def goal_scraper(goal):
    contents, raw_content = [], []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    for i, url in tqdm(enumerate(goal['URL'].tolist()), desc="Processing goal.com URLs"):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            raw_content.append(response.text)

            g = Goose()
            article = g.extract(raw_html=response.text)

            row_id = goal['Document ID'].iloc[i]
            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': article.meta_description,
                    'image': article.top_image.src if article.top_image else None,
                    'domain': article.domain,
                    'title': article.title,
                    'cleaned_text': article.cleaned_text,
                    'opengraph': article.opengraph,
                    'tags': article.tags,
                    'tweets': article.tweets,
                    'movies': article.movies,
                    'links': article.links,
                    'authors': article.authors,
                    'publish_date': str(article.publish_date) if article.publish_date else None
                }
            }

            contents.append(article_data)

        except requests.exceptions.RequestException as e:
            print(f"\nErreur de requête pour {url} : {e}")
            # Pour ne pas décaler les ID, on ajoute un article avec des valeurs par défaut (None)
            row_id = goal['Document ID'].iloc[i]
            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': url},
                    'image': None,
                    'domain': None,
                    'title': None,
                    'cleaned_text': None,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None
                }
            }
            contents.append(article_data)

        except Exception as e:
            # Gestion des autres erreurs
            print(f"\nErreur d'extraction pour {url} : {e}")
            # Même traitement que pour les erreurs de requête
            row_id = goal['Document ID'].iloc[i]
            article_data = {
                'row_id': row_id,
                'data': {
                    'meta': {'canonical': url},
                    'image': None,
                    'domain': None,
                    'title': None,
                    'cleaned_text': None,
                    'opengraph': {},
                    'tags': [],
                    'tweets': [],
                    'movies': [],
                    'links': [],
                    'authors': [],
                    'publish_date': None
                }
            }
            contents.append(article_data)

        time.sleep(0.5)  # Délai pour éviter de surcharger le serveur

    return contents, raw_content


## Gather all 

In [41]:
#Lecture du df meltwater, pour chaque url soit on scrapp normal (selenium) soit on scrapped avec la fonction ad&quate en fonction de la source 
# Scrapping puis enregistrement dans /content (pour le fichier source) et /clean_content (pour le json output)
# Chaque algo de scrap manuel renvoie une liste de json correspnod à chacun de ses articles 

In [ ]:
class Scraper:
    def __init__(self, data, manualy_scrapped_sources):
        self.data = data
        self.manualy_scrapped_sources = manualy_scrapped_sources
        self.source_domains = data['Source Domain'].unique()
        self.minister = None
        

        sys.stdout = open('output.log', 'w')
        print("Everything from now on goes to the log file.")

    def route_scrap(self):
        # Supprimer les dossiers 'content' et 'cleaned_content' s'ils existent
        content_folder = 'content'
        cleaned_content_folder = 'cleaned_content'
        
        if os.path.isdir(content_folder):
            shutil.rmtree(content_folder)  # Supprimer le dossier 'content'
            print(f"Le dossier {content_folder} a été supprimé.")
        
        if os.path.isdir(cleaned_content_folder):
            shutil.rmtree(cleaned_content_folder)  # Supprimer le dossier 'cleaned_content'
            print(f"Le dossier {cleaned_content_folder} a été supprimé.")
        
        # Traiter chaque source
        for source in self.source_domains:
            source_data = self.data[self.data['Source Domain'] == source]

            if source in self.manualy_scrapped_sources:
                self.manual_scrap(source_data)
            else:
                self.default_scrap(source_data)

            # Libérer la mémoire en supprimant les données déjà traitées
            self.data = self.data[self.data['Source Domain'] != source].reset_index(drop=True)


    
    def check_condition(self, article):
        if list(article.keys()) == ['row_id','data'] : 
            article = article['data']
        try : 
            if not article : 
                print("not article")
            if len(article['cleaned_text'])<50:
                return False
            #if 'I been blocked' in str(article["infos"]):
            #    return False
            if 'changement de réseau' in str(article['cleaned_text']):
                return False
            if 'cloudflare' in str(article['title']):
                return False
            #if 'cloudflare' in str(article.infos):
            #    return False
            if 'cloudflare' in str(article['cleaned_text']):
                return False
            if '404 Error' in str(article['title']):
                return False
            return True
        except : 
            print("Article non valide pour le check conditions")
            return False
            
    ################################################################################
    ################################################################################
    ################# Use of manual scraping, coded above ##########################
    ################################################################################
    ################################################################################



    def manual_scrap(self, source_data):
        """
        Nos scrapers manuels renvoient deux objets : content (qui est une liste de clean json) et raw content (une liste de fichiers .txt de page source)
        A partir de cela on peut save les articles dans content et /clean_content avec l'id row fourni dans le "content" 
        
        ####
        content.keys() = [row_id, data] où row_id est l'id du document et data est le json d'intérêt 
        type(raw_content) = .txt
        """
        source_name = source_data['Source Domain'].iloc[0]
        print(f"Manual scraping for {source_name}")
        if source_name == 'msn.com' :
            content, raw_content = msn_scraper(source_data)
        elif source_name == 'sauress.com' :
            content,raw_content = sauress_scraper(source_data)
        elif source_name == 'aawsat.com' :
            content,raw_content = aawsat_scraper(source_data)
        elif source_name == 'alriyadh.com' :
            content,raw_content = alriyadh_scraper(source_data)
        elif source_name == 'klyoum.com' :
            content,raw_content = klyoum_scraper(source_data)
        elif source_name == 'spa.gov.sa' :
            content,raw_content = spa_scraper(source_data)
        elif source_name == 'rt.com' or source_name == 'nytimes.com' or source_name == 'cnn.com' or source_name == 'yahoo.com' :
            content,raw_content = rt_nyt_cnn_scraper(source_data)
        elif source_name == 'shafaqna.com' :
            content,raw_content = shafaqna_scraper(source_data)
        elif source_name == 'goal.com' :
            content,raw_content = goal_scraper(source_data)
        elif source_name == 'sohu.com' :
            content,raw_content = sohu_scraper(source_data)
        elif source_name == 'investing.com' :
            content,raw_content = investing_scraper(source_data) 
        else : 
            raise ValueError("Source Domain non valide pour le manual scrapping")
        
        self.minister = source_data['RB Minister'].iloc[0] #on suppose que le minister est le même dans tout le batch de données 
        # Sauvegarder le contenu brut et nettoyé
        self.manual_save_content(raw_content = raw_content, content = content)
        self.manual_save_clean_content(cleaned_content = content)
    
    def manual_save_content(self, raw_content, content):
        """
        Sauvegarder le contenu brut pour une source spécifique
        `raw_content` = liste de fichiers .txt 
        `content` = liste de json où chaque article contient une clé 'row_id'
        """
        
        folder_path =f"content/{self.minister}"
        os.makedirs(folder_path, exist_ok=True)  
        for i, article in enumerate(raw_content):  # Sauvegarder chaque fichier .txt de raw_content
            row_id = content[i]['row_id']  # Utiliser 'row_id' du contenu
            file_name = f"{row_id}.txt"  # Nom du fichier basé sur 'row_id'
            file_path = os.path.join(folder_path, file_name)
            
            with open(file_path, "w", encoding="utf-8") as file:
                file.write(article)
            

    

    
    def manual_save_clean_content(self, cleaned_content):
        """
        Sauvegarder le contenu nettoyé pour une source spécifique au format JSON, fait automatiquement normalement par les algo de scrap manuels
        """
        folder_path = f"cleaned_content/{self.minister}"
        os.makedirs(folder_path, exist_ok=True)  

        for i, article in enumerate(cleaned_content):  
            if not self.check_condition(article=article) :  #possible que le clean json soit vide (champs vides)
                print(f"Article {article} does not meet the cleaning criteria.")
                continue
            row_id = article['row_id'] 
            file_name = f"{row_id}_cleaned.json" 
            file_path = os.path.join(folder_path, file_name)
            
            cleaned_data = article['data']#on ne veux pas row_id 

            try:
                with open(file_path, "w", encoding="utf-8") as out_file:
                    json.dump(cleaned_data, out_file, ensure_ascii=False, indent=4)
            except Exception as e:
                print(f"Error saving cleaned content for {file_name}: {e}")
            



    ################################################################################
    ################################################################################
    ####### Default Scrapping with firas's method with selenium and goose ##########
    ################################################################################
    ################################################################################
    

    def default_save_content(self, source_data, max_retries=2):
        """Appel pour chaque source, et renvoie le folder dans lequel il a enregistrer les contents (utile dans le cas où l'on a plusieurs ministre)
        et le nombre d'articles appended, pour ensuite clean seulement ceux qui nous interesse et pas parcourir tout le folder /content"""
        options = webdriver.ChromeOptions()
        options.add_argument('--disable-blink-features=AutomationControlled')
        options.add_argument('--ignore-ssl-errors=yes')
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--headless')
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')


        driver = webdriver.Chrome(options=options)
        driver.set_page_load_timeout(30)  # Timeout pour le chargement de la page

        article_saved = 0
        folder_path = 'Error'
        if len(source_data) == 0:
            print("Source data is empty. Exiting.")
            driver.quit()
            return folder_path, article_saved
        for index, row in tqdm(source_data.iterrows(), total=source_data.shape[0]):
            
            retries = 0
            while retries < max_retries:


                try:
                    driver.get(row["URL"])

                    time.sleep(3)  # Attendre quelques secondes pour que la page se charge
                    soup = BeautifulSoup(driver.page_source, 'html.parser')

                    self.minister = row['RB Minister']
                    folder_path = f"content/{self.minister}"
                    os.makedirs(folder_path, exist_ok=True)

                    # Sauvegarder le contenu de la page dans un fichier texte
                    with open(f"{folder_path}/{row['Document ID']}.txt", "w", encoding='utf-8') as text_file:
                        text_file.write(str(soup))

                    article_saved +=1
                    break  # Si tout s'est bien passé, sortir de la boucle de réessai


                except Exception as e:
                    print(f"Error processing {row['URL']}: {e}")
                    retries += 1
                    print(f"Tentative {retries} sur {max_retries} pour {row['URL']}")
                    time.sleep(3) 

                    if retries == max_retries:
                        print(f"Échec après {max_retries} tentatives pour {row['URL']}")
                        folder_path = "Error"
                        break  # Passer à l'URL suivante après le nombre maximum de tentatives
                        

        driver.quit()
        return folder_path, article_saved
    

    def default_save_clean_content(self, content_folder_path, n):
        """On récupere les n derniers fichiers saved par la fonction save_content puis on vérifie si ils peuvent être cleané.
        Si oui on le save, sinon on le passe. """
        
        if not os.path.isdir(content_folder_path):
            print(f'Error: {content_folder_path} does not exist or is not a directory.')
            return  #on a aucun article à clean puisque le content_folder == Error ou n'existe pas 

        g = Goose()
        results = []

        for filename in tqdm(os.listdir(content_folder_path)[-n:]):  # On s'intéresse seulement aux n derniers articles sauvegardés
            
            file_path = os.path.join(content_folder_path, filename)

            if not os.path.isfile(file_path):
                print(f'le file path : {file_path} n\'est pas un file path valide')
                continue  
                
            try:
                with open(file_path, "r", encoding="utf-8") as file:
                    html_raw = file.read()
                article = g.extract(raw_html=html_raw)
                if type(article.publish_date) == list:
                    article.publish_date = article.publish_date[0]
                    print("conversion date de liste à date")
                cleaned_data = {
                    'meta': article.meta_description,
                    'image': article.top_image.src if article.top_image else None,
                    'domain': article.domain,
                    'title': article.title,
                    'cleaned_text': article.cleaned_text,
                    'opengraph': article.opengraph,
                    'tags': article.tags,
                    'tweets': article.tweets,
                    'movies': article.movies,
                    'links': article.links,
                    'authors': article.authors,
                    'publish_date': str(article.publish_date) if article.publish_date else None
                }

                if not self.check_condition(article=cleaned_data): 
                    print(f'le fichier {cleaned_data}.txt ne répond pas aux critères de cleaning ...... ')
                    continue  

                

                document_id = os.path.splitext(filename)[0]  # Extraire l'id avant le .txt
                
                output_folder = f"cleaned_content/{self.minister}"
                os.makedirs(output_folder, exist_ok=True)

                output_file = os.path.join(output_folder, f"{document_id}_clean.json")

                with open(output_file, "w", encoding="utf-8") as out_file:
                    json.dump(cleaned_data, out_file, ensure_ascii=False, indent=4)

            except Exception as e:
                print(f"Error processing {filename}: {e}")  # Afficher l'erreur si une exception se produit pendant le traitement
                continue

        print("Cleaning process completed.")


    def default_scrap(self, source_data):
        '''Main fonction pour scrapper des sources de base : on append tous les articles.txt de la source dans /content puis on s'occupe de les cleans'''
        content_folder_path,n = self.default_save_content(source_data)
        self.default_save_clean_content(content_folder_path,n)

In [134]:
data = pd.read_excel("20250428_Reema_Bandar_Al_Saud_Meltwater raw data (2024).xlsx")
scrapped_article = pd.read_excel("12-05-2025-SourceDomainReport.xlsx")
manualy_scraped_sources = scrapped_article[scrapped_article['Scrapped'] == True]['Source Domain'].tolist()
unscrappable = scrapped_article[scrapped_article['Scrapped'] == False]['Source Domain'].tolist()

In [135]:
data['RB Minister'] = ["Reema_Bandar"] * len(data)

In [ ]:
data = data[~data["Source Domain"].isin(unscrappable)]

In [145]:
scraper = Scraper(data =data,manualy_scrapped_sources=manualy_scraped_sources)
scraper.route_scrap()

100%|██████████| 6/6 [00:02<00:00,  2.07it/s]
Processing Alriyadh URLs: 106it [01:15,  1.41it/s]
processing klyoum URLs: 60it [02:13,  2.23s/it]


KeyboardInterrupt: 